In [1]:
using Logging, LinearAlgebra, ProgressMeter, Revise

In [2]:
global_logger(ConsoleLogger(stdout, Logging.Info))
using PorousMaterials, MOFun

[ Info: Precompiling MOFun [top-level]


In [3]:
s_moty = moiety("find-replace/2-!-p-phenylene");

In [4]:
r_moty = moiety("2-acetylamido-p-phenylene");

In [5]:
parent = Crystal("IRMOF-1_one_ring.cif")
strip_numbers_from_atom_labels!(parent)
infer_bonds!(parent, true)

┌ Info: Crystal IRMOF-1_one_ring.cif has  space group. I am converting it to P1 symmetry.
└         To afrain from this, pass `convert_to_p1=false` to the `Crystal` constructor.


In [6]:
# @eval MOFun DEBUG = true
search = s_moty ∈ parent
for i in 1:search.num_isomorphisms
    xtal = find_replace(s_moty, r_moty, parent, c=i, s=search)
    write_cif(xtal, "output/$(parent.name)/" * xtal.name * "_$i.cif")
end
@eval MOFun DEBUG = false;

In [7]:
parent = Crystal("IRMOF-1.cif")
strip_numbers_from_atom_labels!(parent)
infer_bonds!(parent, true)

In [8]:
# @eval MOFun DEBUG = true
search = s_moty ∈ parent
@showprogress 1 "Processing replacements:\t" for i in 1:search.num_isomorphisms
    xtal = find_replace(s_moty, r_moty, parent, c=i, s=search)
    write_cif(xtal, "output/$(parent.name)/" * xtal.name * "_$i.cif")
end
@eval MOFun DEBUG = false;

Processing replacements:	100%|██████████████████████████| Time: 0:05:07


### result 5 crosses a periodic boundary

In [9]:
write_xyz(parent[search.results[5].isomorphism], "output/result5")

In [11]:
@eval MOFun DEBUG = true
find_replace(s_moty, r_moty, parent, c=5, s=search)
@eval MOFun DEBUG = false;